In [1]:
from utils import RL_with_constraint, RL_without_constraint, RLWKC_with_constraint, RLWKC_without_constraint, HTP, sparsePermutation, addNoise, meanNormalizedError, sgn, one_norm, only_priors
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from IPython.display import clear_output
import random  
import scipy
np.random.seed(90)
random.seed(90)

In [2]:
n_Permuted = 16                                           # number of permutation
cv_measurements = 5
m = 32                                                    # number of known correspondences
noise_fraction = 0.02
permutation_runs, noise_runs = 2, 2
p = 240                                                   # dimension of x
k = 14                                                    # sparsity of x
N_Metrics = np.zeros(shape = (permutation_runs, noise_runs, 9))
x = np.load("x_sparsity_" + str(k) + ".npy")
A_cv =  np.load("A_cv_fixed.npy")  
y_cv = addNoise(A_cv @ x, noise_fraction)     


for N in [110]:
    A = np.load("A_fixed.npy")[:N, :]           # extract N rows from the generated matrix

    for per_run in range(permutation_runs):
        print("N: {}, Permutation run: {}".format(N, per_run))
        P = sparsePermutation(N, m, n_Permuted)
        noiseLess_y = P @ A @ x
        z_true = noiseLess_y - A @ x
        max_abs_z = np.max(np.abs(z_true))
        sum_abs_z = np.sum(np.abs(z_true))

        l1_RLWKC_with_constraint, l2_RLWKC_with_constraint = None, None
        l1_RLWKC_without_constraint, l2_RLWKC_without_constraint = None, None
        l1_RL_with_constraint, l2_RL_with_constraint = None, None
        l1_RL_without_constraint, l2_RL_without_constraint = None, None
        l1_one_norm = None
        l1_only_priors = None

        for noise_run in range(noise_runs):
            print("N: {}, Noisy run: {}".format(k, noise_run))
            noisy_y = addNoise(noiseLess_y, noise_fraction)
            xhat_RLWKC_with_constraint, l1_RLWKC_with_constraint, l2_RLWKC_with_constraint = RLWKC_with_constraint(A, noisy_y, A_cv, y_cv, m, l1_RLWKC_with_constraint, l2_RLWKC_with_constraint)
            xhat_RLWKC_without_constraint, l1_RLWKC_without_constraint, l2_RLWKC_without_constraint = RLWKC_without_constraint(A, noisy_y, A_cv, y_cv, m, l1_RLWKC_without_constraint, l2_RLWKC_without_constraint)
            xhat_RL_with_constraint, l1_RL_with_constraint, l2_RL_with_constraint = RL_with_constraint(A, noisy_y, A_cv, y_cv, l1_RL_with_constraint, l2_RL_with_constraint)
            xhat_RL_without_constraint, l1_RL_without_constraint, l2_RL_without_constraint = RL_without_constraint(A, noisy_y, A_cv, y_cv, l1_RL_without_constraint, l2_RL_without_constraint)
            xhat_HTP = HTP(np.concatenate((A, A_cv), axis = 0), np.concatenate((noisy_y, y_cv), axis = 0), k)
            xhat_one_norm, l1_one_norm = one_norm(A, noisy_y, A_cv, y_cv, l1_one_norm)
            xhat_only_priors, l1_only_priors = only_priors(A, noisy_y, A_cv, y_cv, m, l1_only_priors)
  
            N_Metrics[per_run][noise_run][0] = meanNormalizedError(x, xhat_RLWKC_with_constraint)
            N_Metrics[per_run][noise_run][1] = meanNormalizedError(x, xhat_RLWKC_without_constraint)
            N_Metrics[per_run][noise_run][2] = meanNormalizedError(x, xhat_RL_with_constraint)
            N_Metrics[per_run][noise_run][3] = meanNormalizedError(x, xhat_RL_without_constraint)
            N_Metrics[per_run][noise_run][4] = meanNormalizedError(x, xhat_HTP)
            N_Metrics[per_run][noise_run][5] = meanNormalizedError(x, xhat_one_norm)
            N_Metrics[per_run][noise_run][6] = meanNormalizedError(x, xhat_only_priors)
            N_Metrics[per_run][noise_run][7] = max_abs_z
            N_Metrics[per_run][noise_run][8] = sum_abs_z
           

            print(N_Metrics[per_run][noise_run][0], N_Metrics[per_run][noise_run][1], N_Metrics[per_run][noise_run][2], N_Metrics[per_run][noise_run][3], N_Metrics[per_run][noise_run][4], N_Metrics[per_run][noise_run][5], N_Metrics[per_run][noise_run][6], N_Metrics[per_run][noise_run][7], N_Metrics[per_run][noise_run][8])

    np.save('N_Metrics' + str(N) + '.npy', N_Metrics)

N: 110, Permutation run: 0
N: 14, Noisy run: 0
Choose lambda1 properly
0.023851696051263284 0.035433232050466314 0.04733905074862231 0.04770500371402459 0.7204348076770984 0.877710929290953 0.7243003278870811 14.258056767165815 94.90914454295554
N: 14, Noisy run: 1
Choose lambda1 properly
0.029181851405556875 0.04612223774351235 0.0570977548307005 0.059646642877535634 0.7301647118167068 0.893228891545938 0.7007339941698228 14.258056767165815 94.90914454295554
N: 14, Noisy run: 2
Choose lambda1 properly
0.038727094144648665 0.042478818054964 0.06551314972160657 0.06870097090642018 0.6013678905609385 0.8684351864015443 0.7395750653453361 14.258056767165815 94.90914454295554
N: 14, Noisy run: 3
Choose lambda1 properly
0.031909988778198624 0.03921807983249676 0.05414936839191201 0.05279304179226126 0.6033035703979026 0.8832567775099603 0.7041499533353258 14.258056767165815 94.90914454295554
N: 14, Noisy run: 4
Choose lambda1 properly
0.04292643919861003 0.05149722176069095 0.09959403026182

In [ ]:
print(x)

[[-0.22448667]
 [-0.22752394]
 [-0.59139893]
 [-0.79922765]
 [ 1.1901095 ]
 [-1.36589487]
 [ 0.7209559 ]
 [-1.32556175]
 [ 1.85693165]
 [ 1.24718034]
 [-1.35425836]
 [-0.74047347]
 [ 0.23407543]
 [-1.36223476]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     